In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.optimize import minimize, differential_evolution

In [75]:
#加载数据
df = pd.read_excel('各参数SEA.xlsx')
# print(df.sample(10))

In [76]:
#描述性统计
print(df.describe())
print(sea_range := df['SEA'].max() - df['SEA'].min())

             α(°)       β(°)       l1/l2        t_mm          SEA
count  560.000000  560.00000  560.000000  560.000000   560.000000
mean    45.000000   55.00000    2.250000    1.150000   669.679696
std     10.008941    7.07739    0.559517    0.111903   151.469724
min     30.000000   45.00000    1.500000    1.000000   368.340000
25%     35.000000   50.00000    1.875000    1.075000   544.565000
50%     45.000000   55.00000    2.250000    1.150000   684.160000
75%     55.000000   60.00000    2.625000    1.225000   785.387500
max     60.000000   65.00000    3.000000    1.300000  1023.070000
654.73


In [77]:
#数据拆分
X_train, X_test, y_train, y_test = train_test_split(
    np.array(df[['α(°)',  'β(°)',  'l1/l2',  't_mm',]]),
    np.array(df[['SEA']]),
    test_size=0.2,
    shuffle=True,
    random_state=42
)

In [78]:
#训练梯度提升树模型
model = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    min_samples_split=20,
    min_samples_leaf=10,
    subsample=0.8,
    random_state=42,
)
model.fit(X_train, y_train.ravel())

,loss,'squared_error'
,learning_rate,0.1
,n_estimators,100
,subsample,0.8
,criterion,'friedman_mse'
,min_samples_split,20
,min_samples_leaf,10
,min_weight_fraction_leaf,0.0
,max_depth,5
,min_impurity_decrease,0.0
,init,None


In [79]:
#模型评估
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2_test = r2_score(y_test, y_pred)
r2_train = r2_score(y_train, model.predict(X_train))
print(rmse)
print(r2_test)
print(r2_train)

20.808039175167856
0.9823049971042702
0.9899993007086643


In [80]:
#优化
##定义目标函数
def objective(x):
    x = np.array(x).reshape(1, -1)
    y = model.predict(x)
    return -y

In [81]:
##定义边界约束
param_bounds = []
for i in df.iloc[:, 0:4]:
    param_bounds.append((df[i].min(), df[i].max()))
# print(param_bounds)

In [82]:
##differential_evolution
result = differential_evolution(
        objective,
        bounds=param_bounds,
        strategy='best1bin',
        maxiter=1000,
        popsize=15,
        tol=0.01,
        mutation=(0.5, 1),
        seed=42,
        disp=True,
    )
optimal_params = result.x
optimal_sea = -result.fun
print(optimal_params)
print(optimal_sea)

differential_evolution step 1: f(x)= -943.1560575897082
differential_evolution step 2: f(x)= -956.3301078186308
differential_evolution step 3: f(x)= -978.1619779716162
differential_evolution step 4: f(x)= -978.1619779716162
differential_evolution step 5: f(x)= -978.1619779716162
differential_evolution step 6: f(x)= -978.1619779716162
differential_evolution step 7: f(x)= -978.1619779716162
differential_evolution step 8: f(x)= -978.1619779716162
differential_evolution step 9: f(x)= -978.1619779716162
differential_evolution step 10: f(x)= -978.1619779716162
differential_evolution step 11: f(x)= -978.1619779716162
differential_evolution step 12: f(x)= -978.1619779716162
Polishing solution with 'L-BFGS-B'
[36.78864001 61.52113281  1.58033276  1.27145852]
978.1619779716162
